### Summary:

This is a QnA service implementation using 

    # Simple Transformers (https://github.com/ThilinaRajapakse/simpletransformers) library and 
    # DistilBERT language model (https://github.com/huggingface/transformers/tree/master/examples/distillation)

In [1]:
#!pip install simpletransformers

In [2]:
from simpletransformers.question_answering import QuestionAnsweringModel
import os
import json

In [3]:
# Create dummy data to use for training.
train_data = [
    {
        'context': "This is the first context",
        'qas': [
            {
                'id': "00001",
                'is_impossible': False,
                'question': "Which context is this?",
                'answers': [
                    {
                        'text': "the first",
                        'answer_start': 8
                    }
                ]
            }
        ]
    }
]

In [4]:
# save as json file
os.makedirs('simpletransformers-training-data', exist_ok=True)
with open('simpletransformers-training-data/train.json','w') as f:
    json.dump(train_data, f)

In [6]:
# create QuestionAnsweringMode
qnaModel = QuestionAnsweringModel('distilbert','distilbert-base-uncased-distilled-squad', args={'reprocess_input_data': True, 'overwrite_output_dir': True},use_cuda=False)

In [7]:
# train the model
qnaModel.train_model('simpletransformers-training-data/train.json')

Converting to features started.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1001.98it/s]


Running loss: 0.819692
Training of distilbert model complete. Saved to outputs/.


In [8]:
# evaluate the model
result, text = qnaModel.eval_model('simpletransformers-training-data/train.json')
print (result)
print (text)

Converting to features started.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 998.41it/s]



[{'context': 'This is the first context', 'qas': [{'id': '00001', 'is_impossible': False, 'question': 'Which context is this?', 'answers': [{'text': 'the first', 'answer_start': 8}]}]}]
{'correct': 1, 'similar': 0, 'incorrect': 0}
{'correct_text': {'00001': 'the first'}, 'similar_text': {}, 'incorrect_text': {}}


In [21]:
# prediction using the model
to_predict=[{
    'context': "lex Wright has 13 years of investment experience. He joined Fidelity in 2001 as a European equity research analyst, successively covering building materials, alcoholic beverages, leisure, emerging European and African banks and UK small-cap stocks. He became portfolio manager of the Fidelity UK Smaller Companies Fund in 2008. He continues to manage this fund alongside the Fidelity Special Situations Fund and the Fidelity Special Values PLC, which he started managing in 2012.",
    'qas': [{
        'question': "Who managed fidelity special situations fund",'id':'0', ## Right
        'question': "When did lex joined fidelity",'id':'1', ## Right
        'question': "which role did lex joined fidelity",'id':'2', ## Right
        'question': "how many funds does lex manages",'id':'3'  ##Wrong
    }]
}]
prediction = qnaModel.predict(to_predict)
print (prediction)

Converting to features started.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 249.47it/s]



[{'id': '3', 'answer': '13 years of investment experience'}]
